In [ ]:
#import scipy.spatial.distance as sdist
import umap
from sklearn import preprocessing
import time
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, fcluster
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#import math
import os
import random
#import requests
#from BeautifulSoup import BeautifulSoup
import matplotlib.lines as mlines
import matplotlib as mpl
from pylab import *
#from ete3 import NCBITaxa
#ncbi = NCBITaxa()
import json
from ast import literal_eval
from matplotlib.cm import ScalarMappable
import scipy
import scipy.stats
import scipy.signal
from matplotlib.ticker import MaxNLocator
strain_color_list = ["#33bbff","#0050e6","#009999", "#777777"]
umap_plot_shape = ['D','o','s','v']
default_color_list = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b', u'#e377c2', u'#7f7f7f', u'#bcbd22', u'#17becf']

In [ ]:
# function 
# input is query_cluster, qseqid, qstart and qend
# output is to print out corresponding read
def get_sequence(assembly_dirt, assembly_file_name, qseqid, qstart, qend):
    with open(assembly_dirt + assembly_file_name) as finput:
        while True:
            line_read = finput.readline()  
            if len (line_read) == 0:
                break
            if ">" in line_read:
                if qseqid in line_read:
                    line_read = ''
                    while True:
                        line_read_temp = finput.readline().strip()
                        line_read = line_read+line_read_temp
                        if len (line_read) == 0 or ">" in line_read:
                            break
                    string_of_mobile_gene = line_read.strip()[qstart-1:qend]
                    #print len(string_of_mobile_gene),string_of_mobile_gene
                    return string_of_mobile_gene
                    break

In [ ]:
# a function to take intervals and return merged intervals
def merge_intervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            # test for intersection between lower and higher:
            # we know via sorting that lower[0] <= higher[0]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = (lower[0], upper_bound)  # replace by merged interval
            else:
                merged.append(higher)
    return merged

In [ ]:
HGT_strain_20210302_dirt = '/media/z/feb2021/microbe_seq_analysis/\
Q5_variant_calling_100fewer/strain_HGT_high_quality_20210302/'
# this folder contains all the high-quality, strain-resovled genomes

In [ ]:
%%capture
#### this is to make database for all cleaned high quality genomes
#### get name list
file_list_of_assemblies = os.listdir(HGT_strain_20210302_dirt)
for file_temp in file_list_of_assemblies:
    if '.fna' not in file_temp:
        file_list_of_assemblies.remove(file_temp)
#%cd {HGT_strain_20210302_dirt}
for file_temp in file_list_of_assemblies:
    !makeblastdb -dbtype "nucl" -in {HGT_strain_20210302_dirt}{file_temp} \
    -out {HGT_strain_20210302_dirt}{file_temp[:-4]}_blastdb

In [ ]:
#### this is to blast each against all other reference genomes
for query_temp in file_list_of_assemblies:
    for subject_temp in file_list_of_assemblies:
        !blastn -query {HGT_strain_20210302_dirt}{query_temp} -db \
        {HGT_strain_20210302_dirt}{subject_temp[:-4]}_blastdb \
        -outfmt 6 -perc_identity 99.98 \
        -out {HGT_strain_20210302_dirt}{query_temp[:-4]}_to_{subject_temp[:-4]}_blast_result

In [ ]:
# this is to read in all blastn result and put them in df

# First get all file names
file_list_of_assemblies = os.listdir(HGT_strain_20210302_dirt)
a_temp_list = []
blastn_HGT_result = []
for i in range(len(file_list_of_assemblies)):
    file_temp = file_list_of_assemblies[i]
    if '_blast_result' not in file_temp:
        continue
    elif file_temp.split('to')[0][:-1] != \
    file_temp.split('to')[1][1:1+len(file_temp.split('to')[0][:-1])]:
        a_temp_list += [file_temp]
file_list_of_assemblies = a_temp_list
print(len(file_list_of_assemblies))

for file_temp in file_list_of_assemblies:
    try:
        temp_blastn_result = pd.read_csv(HGT_strain_20210302_dirt+file_temp,\
                                         header = None, sep = '\t')
    except:
        #print file_temp, 'has no dection'
        continue            
    temp_blastn_result.columns = \
    ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', \
     'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
    temp_blastn_result['query_cluster'] = [file_temp.split('to')[0][:-1]]*len(temp_blastn_result)
    temp_blastn_result['subject_cluster'] = [file_temp.split('to')[1][1:-len('_blast_result')]]\
    *len(temp_blastn_result)
    temp_blastn_result = temp_blastn_result.loc[temp_blastn_result['length']>500]
    if len(blastn_HGT_result) > 0: # if not the first file, than cat dataframes
        blastn_HGT_result = pd.concat([blastn_HGT_result, temp_blastn_result])
    else:# if this is the first file with information, than apply the length to it
        blastn_HGT_result = temp_blastn_result        
blastn_HGT_result = blastn_HGT_result.sort_values(by=['query_cluster', 'subject_cluster',])
blastn_HGT_result = blastn_HGT_result.reset_index(drop=True)
cols = list(blastn_HGT_result.columns.values)
blastn_HGT_result = blastn_HGT_result[cols[-2:-1] + cols[:1] + cols[-1:] + cols[1:2] + cols[2:-2]]
blastn_HGT_result.to_csv(HGT_strain_20210302_dirt+'blastn_HGT_result.csv', index=False)

In [ ]:
length_threshold = 5000
match_bp_ratio = 99.98
blastn_HGT_result = pd.read_csv(HGT_strain_20210302_dirt+'blastn_HGT_result.csv')
blastn_result_sc = blastn_HGT_result.loc[blastn_HGT_result['length']>length_threshold].\
loc[blastn_HGT_result.loc[blastn_HGT_result['length']>length_threshold]['pident']>match_bp_ratio]
blastn_result_sc = blastn_result_sc.reset_index(drop=True)
blastn_result_sc.to_csv(HGT_strain_20210302_dirt+'blastn_HGT_result_trimmed.csv', index=False)

In [ ]:
query_cluster_list = []
subject_cluster_list = []
total_length_of_HGT = []
blastn_result_sc = pd.read_csv(HGT_strain_20210302_dirt+'blastn_HGT_result_trimmed.csv')
assembly_all = list(set(list(set(blastn_result_sc['query_cluster']))+\
                        list(set(blastn_result_sc['subject_cluster']))))
for query_temp in assembly_all:    
    df_temp_query = blastn_result_sc.loc[blastn_result_sc['query_cluster']==query_temp]
    for subject_temp in list(set(df_temp_query['subject_cluster'])):
        df_temp_query_subject = df_temp_query.loc[df_temp_query['subject_cluster']==subject_temp]
        df_temp_query_subject = df_temp_query_subject.reset_index(drop=True)
        query_cluster_list += [query_temp]
        subject_cluster_list += [subject_temp]
        all_contig_list_temp = list(set(df_temp_query_subject['qseqid']))
        total_HGT_count = 0
        for i in range(len(all_contig_list_temp)):
            contig_to_check_temp = all_contig_list_temp[i]
            df_temp_query_subject_contig = df_temp_query_subject.loc\
            [df_temp_query_subject['qseqid']==contig_to_check_temp]
            df_temp_query_subject_contig = df_temp_query_subject_contig.reset_index(drop=True)
            all_intervals_temp = []
            for j in range(len(df_temp_query_subject_contig)):
                start_temp = min(df_temp_query_subject_contig['qstart'][j], df_temp_query_subject_contig['qend'][j])
                end_temp = max(df_temp_query_subject_contig['qstart'][j], df_temp_query_subject_contig['qend'][j])
                all_intervals_temp += [(start_temp,end_temp)]
            merged_intervals_temp = merge_intervals(all_intervals_temp)
            total_length_invervals = 0
            for each_interval_temp in merged_intervals_temp:
                total_length_invervals += each_interval_temp[1]-each_interval_temp[0] + 1
            total_HGT_count += total_length_invervals
        total_length_of_HGT += [total_HGT_count]
blastn_result_sc_bp_count = pd.DataFrame({'query_cluster_list':query_cluster_list})
blastn_result_sc_bp_count['subject_cluster_list'] = subject_cluster_list
blastn_result_sc_bp_count['total_length_of_HGT'] = total_length_of_HGT
blastn_result_sc_bp_count.to_csv(HGT_strain_20210302_dirt+'blastn_result_sc_bp_count.csv', index = False)